In [15]:
# create_individual_files.py
#!/usr/bin/env python3
import os
import shutil
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment
from openpyxl.utils import get_column_letter

# === Einstellungen ===
FONT = Font(name='Arial', size=8)
FONT_BOLD = Font(name='Arial', size=8, bold=True)
thin = Side(border_style='thin', color='000000')
border = Border(left=thin, right=thin, top=thin, bottom=thin)
group_ranges = [('Time (μs)', 2, 4), ('Heap (B)', 5, 7), ('Stack (B)', 8, 10)]

# Sortierreihenfolge für Modelle
MODEL_ORDER = ['Pi4b', 'Pi3b', 'PiZero2', 'PiBanana', 'PiZero', 'Pi1bPlus', 'Pi1b']

# 1. Pfade definieren
csv_input = 'output/csv/nist_all.csv'
output_dir = 'output/xlsx/nist'

# 2. Output-Verzeichnis bereinigen
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# 3. CSV einlesen und Spaltennamen vereinheitlichen
df = pd.read_csv(csv_input)
df.columns = df.columns.str.lower()

# 4. Einzeldateien pro Algorithmus erstellen
for algo, grp in df.groupby('algorithmus'):
    safe = algo.replace('/', '_').replace(' ', '_')
    file_path = os.path.join(output_dir, f'{safe}.xlsx')
    wb = Workbook()
    ws = wb.active
    ws.title = 'NIST All'

    # Runden-Wert
    runden = grp['runden']
    runden_wert = int(runden.iat[0]) if runden.nunique() == 1 else 'Var'
    c = ws.cell(row=1, column=1, value=f"R = {runden_wert}")
    c.font = FONT
    c.alignment = Alignment(horizontal='center', vertical='center')
    c.border = border

    # Typ und Mapping (kem oder sig)
    if 'typ' in grp.columns and grp['typ'].notna().any():
        typ_algo = str(grp['typ'].dropna().iloc[0]).lower()
    else:
        typ_algo = 'kem'
    subtitle_map_algo = {'Enc': 'Sign', 'Dec': 'Verify'} if typ_algo == 'sig' else {'Enc': 'Enc', 'Dec': 'Dec'}

    # Flaches DataFrame
    df_flat = pd.DataFrame({
        'model':        grp['model'],
        'keygen_time':  grp['keygen_time'],
        'enc_time':     grp['enc_time'],
        'dec_time':     grp['dec_time'],
        'keygen_heap':  grp['keygen_heap'],
        'enc_heap':     grp['enc_heap'],
        'dec_heap':     grp['dec_heap'],
        'keygen_stack': grp['keygen_stack'],
        'enc_stack':    grp['enc_stack'],
        'dec_stack':    grp['dec_stack'],
    })

    # Sortiere nach Modellreihenfolge
    df_flat['model'] = pd.Categorical(df_flat['model'], categories=MODEL_ORDER, ordered=True)
    df_flat = df_flat.sort_values('model')

    # Dynamische Spaltenbeschriftungen
    col_labels = ['Model']
    for _ in range(3):
        col_labels += ['Key', subtitle_map_algo['Enc'], subtitle_map_algo['Dec']]
    df_flat.columns = col_labels

    # Header-Gruppen
    for title, start, end in group_ranges:
        ws.merge_cells(start_row=1, start_column=start, end_row=1, end_column=end)
        # Rahmen für alle Zellen im Merge-Bereich setzen
        for col in range(start, end+1):
            cell = ws.cell(row=1, column=col)
            if col == start:
                cell.value     = title
                cell.font      = FONT_BOLD
                cell.alignment = Alignment(horizontal='center', vertical='center')
            cell.border    = border

    # Untertitelreihe
    for idx, header in enumerate(col_labels, start=1):
        c = ws.cell(row=2, column=idx, value=header)
        c.font = FONT_BOLD
        c.alignment = Alignment(horizontal='center', vertical='center')
        c.border = border

    # Daten eintragen
    for r, row in enumerate(df_flat.itertuples(index=False), start=3):
        for c_idx, val in enumerate(row, start=1):
            cell = ws.cell(row=r, column=c_idx, value=val)
            cell.font = FONT
            cell.border = border

    # Spaltenbreiten anpassen (nur Header+Daten, ohne erste Merge-Zeile)
    for col_idx in range(1, len(col_labels) + 1):
        letter = get_column_letter(col_idx)
        max_len = 0
        for row in range(2, ws.max_row + 1):
            cell = ws.cell(row=row, column=col_idx)
            if cell.value is None:
                continue
            val = cell.value
            # Wenn float ohne Nachkommastellen, als int darstellen
            if isinstance(val, float) and val.is_integer():
                text = str(int(val))
            else:
                text = str(val)
            max_len = max(max_len, len(text))
        ws.column_dimensions[letter].width = max_len + 1

    # Zeilenhöhe reduzieren (alle Zeilen)
    for r in range(1, ws.max_row + 1):
        ws.row_dimensions[r].height = 12
        
    wb.save(file_path)
    print(f"Einzeldatei für {algo}: {file_path}")


Einzeldatei für BIG_QUAKE_1: output/xlsx/nist\BIG_QUAKE_1.xlsx
Einzeldatei für BIG_QUAKE_3: output/xlsx/nist\BIG_QUAKE_3.xlsx
Einzeldatei für BIG_QUAKE_5: output/xlsx/nist\BIG_QUAKE_5.xlsx
Einzeldatei für CRYSTALS-DILITHIUM_mode2_AES: output/xlsx/nist\CRYSTALS-DILITHIUM_mode2_AES.xlsx
Einzeldatei für CRYSTALS-DILITHIUM_mode2_SHA: output/xlsx/nist\CRYSTALS-DILITHIUM_mode2_SHA.xlsx
Einzeldatei für CRYSTALS-DILITHIUM_mode3_AES: output/xlsx/nist\CRYSTALS-DILITHIUM_mode3_AES.xlsx
Einzeldatei für CRYSTALS-DILITHIUM_mode3_SHA: output/xlsx/nist\CRYSTALS-DILITHIUM_mode3_SHA.xlsx
Einzeldatei für CRYSTALS-DILITHIUM_mode5_AES: output/xlsx/nist\CRYSTALS-DILITHIUM_mode5_AES.xlsx
Einzeldatei für CRYSTALS-DILITHIUM_mode5_SHA: output/xlsx/nist\CRYSTALS-DILITHIUM_mode5_SHA.xlsx
Einzeldatei für Ding_LWE: output/xlsx/nist\Ding_LWE.xlsx
Einzeldatei für EMBLEM-EMBLEM: output/xlsx/nist\EMBLEM-EMBLEM.xlsx
Einzeldatei für EMBLEM-R_EMBLEM: output/xlsx/nist\EMBLEM-R_EMBLEM.xlsx
Einzeldatei für Falcon_1024_fpu: o